In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import pandas as pd
from catboost import CatBoostRegressor
        
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

train = pd.read_csv('/kaggle/input/agriyield-2025/train.csv')
test= pd.read_csv('/kaggle/input/agriyield-2025/test.csv')

x_train = train.drop(columns=["field_id", "yield"])
y_train = train["yield"]

x_test = test.drop(columns=["field_id"])

# Split train data into training and validation
x_train_split, x_val, y_train_split, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Train CatBoost on training split
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    eval_metric='RMSE',
    random_seed=42,
    verbose=200
)

model.fit(x_train_split, y_train_split, eval_set=(x_val, y_val), verbose=200)

# Predictions on validation set
y_val_pred = model.predict(x_val)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print("Validation RMSE:", rmse)
print("Validation MAE:", mae)
print("Validation R²:", r2)


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# ================== Train Huber Regressor ==================
model = HuberRegressor(epsilon=1.35, max_iter=1000)
model.fit(x_train, y_train)

# ================== Validation Predictions & Metrics ==================
y_val_pred = model.predict(x_val)

mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = mse_val ** 0.5
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

print("=== Validation Performance ===")
print(f"MSE: {mse_val:.3f}")
print(f"RMSE: {rmse_val:.3f}")
print(f"MAE: {mae_val:.3f}")
print(f"R2: {r2_val:.3f}")

# ================== Test Predictions & Metrics (if y_test exists) ==================
if 'yield' in test.columns:
    y_test_true = test['yield']
    y_test_pred = model.predict(x_test)
    
    mse_test = mean_squared_error(y_test_true, y_test_pred)
    rmse_test = mse_test ** 0.5
    mae_test = mean_absolute_error(y_test_true, y_test_pred)
    r2_test = r2_score(y_test_true, y_test_pred)

    print("\n=== Test Performance ===")
    print(f"MSE: {mse_test:.3f}")
    print(f"RMSE: {rmse_test:.3f}")
    print(f"MAE: {mae_test:.3f}")
    print(f"R2: {r2_test:.3f}")
else:
    # Predict on test dataset for submission
    y_test_pred = model.predict(x_test)

# ================== Create Submission CSV ==================
submission = pd.DataFrame({
    "field_id": test["field_id"],  # assuming 'field_id' exists in test dataset
    "yield": y_test_pred
})

submission.to_csv("huber_submission.csv", index=False)
print("\nSubmission file 'huber_submission.csv' created successfully.")


In [ ]:
from sklearn.linear_model import HuberRegressor
import lightgbm as lgb

# Huber model
huber_model = HuberRegressor(epsilon=1.35, max_iter=1000)
huber_model.fit(x_train, y_train)
y_val_huber = huber_model.predict(x_val)
y_test_huber = huber_model.predict(x_test)

# LightGBM model
lgb_model = lgb.LGBMRegressor(
    n_estimators=1000, learning_rate=0.05, max_depth=-1,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)
lgb_model.fit(x_train, y_train, eval_set=[(x_val, y_val)],
              eval_metric="l2", callbacks=[lgb.early_stopping(stopping_rounds=50)])
y_val_lgb = lgb_model.predict(x_val)
y_test_lgb = lgb_model.predict(x_test)


In [ ]:
# Validation ensemble
y_val_ensemble = 0.3 * y_val_huber + 0.7 * y_val_lgb

# Test ensemble
y_test_ensemble = 0.3 * y_test_huber + 0.7 * y_test_lgb


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_val, y_val_ensemble)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_val_ensemble)
r2 = r2_score(y_val, y_val_ensemble)

print(f"Ensemble Performance - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")


In [ ]:
submission = pd.DataFrame({
    "field_id": test["field_id"],
    "yield": y_test_ensemble
})
submission.to_csv("ensemble_submission.csv", index=False)
print("Ensemble submission saved.")


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Initialize model
gbr_model = GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    random_state=42
)

# Train model
gbr_model.fit(x_train, y_train)

# Predict on validation set
y_val_pred = gbr_model.predict(x_val)

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Gradient Boosting Performance - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")


In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Initialize OMP model
omp_model = OrthogonalMatchingPursuit(n_nonzero_coefs=5)  # You can adjust n_nonzero_coefs

# Train model
omp_model.fit(x_train, y_train)

# Predict on validation set
y_val_pred = omp_model.predict(x_val)

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Orthogonal Matching Pursuit Performance - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# ================== Initialize Gradient Boosting ==================
gbr_model = GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    random_state=42
)

# ================== Train on training data ==================
gbr_model.fit(x_train, y_train)

# ================== Validate on validation set ==================
y_val_pred = gbr_model.predict(x_val)

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Gradient Boosting Performance - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")

# ================== Predict on test dataset ==================
y_test_pred = gbr_model.predict(x_test)

# ================== Create submission file ==================
submission = pd.DataFrame({
    "field_id": test["field_id"],  # ensure this column exists in test dataset
    "yield": y_test_pred
})

submission.to_csv("gradient_boosting_submission.csv", index=False)
print("Submission file 'gradient_boosting_submission.csv' created successfully.")


In [ ]:
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# ================== Initialize Poisson Regressor ==================
poisson_model = PoissonRegressor(
    alpha=1.0,          # regularization strength
    max_iter=1000,
    fit_intercept=True
)

# ================== Train on training data ==================
poisson_model.fit(x_train, y_train)

# ================== Validate on validation set ==================
y_val_pred = poisson_model.predict(x_val)

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Poisson Regressor Performance - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")

# ================== Predict on test dataset ==================
y_test_pred = poisson_model.predict(x_test)

# ================== Create submission file ==================
submission = pd.DataFrame({
    "field_id": test["field_id"],  # ensure this column exists in test dataset
    "yield": y_test_pred
})

submission.to_csv("poisson_submission.csv", index=False)
print("Submission file 'poisson_submission.csv' created successfully.")


In [ ]:
import pandas as pd
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import files  # or not needed if running in Kaggle



# ================= Orthogonal Matching Pursuit =================
omp_model = OrthogonalMatchingPursuit(n_nonzero_coefs=5)  # you can tune n_nonzero_coefs
omp_model.fit(x_train, y_train)

# Predict on validation set
y_val_pred = omp_model.predict(x_val)

# Metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print("Orthogonal Matching Pursuit Performance")
print(f"MSE: {mse:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

# ================= Predict on test dataset =================
y_test_pred = omp_model.predict(x_test)

# ================= Create submission =================
submission = pd.DataFrame({
    "field_id": test["field_id"],  # adjust if field_id column exists
    "yield": y_test_pred
})

submission.to_csv("omp_submission.csv", index=False)
print("Submission file 'omp_submission.csv' created successfully.")


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Initialize SVR model
svr_model = SVR(kernel="rbf", C=100, gamma="scale", epsilon=0.1)

# Fit on training split
svr_model.fit(x_train_split, y_train_split)

# Predictions on validation set
y_val_pred = svr_model.predict(x_val)

# Metrics on validation
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print("SVR Model Performance (Validation Set):")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R2: {r2:.4f}")

# ---- Kaggle Submission ----
# Predict on test set (x_test)
y_test_pred = svr_model.predict(x_test)

submission = pd.DataFrame({
    "field_id": test["field_id"],   # keep IDs from test
    "yield": y_test_pred            # predicted yield
})

submission.to_csv("svr_submission.csv", index=False)
print("✅ SVR submission file created: svr_submission.csv")


